In [34]:
import pandas as pd
import numpy as np

import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

## **СЕРИАЛИЗАЦИЯ И ДЕСЕРИАЛИЗАЦИЯ**

In [3]:
# Загружаем датасет о диабете
X, y = load_diabetes(return_X_y=True)
# Инициализируем модель линейной регрессии
regressor = LinearRegression().fit(X,y)

## LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [5]:
# Производим сериализацию обученной модели
model = pickle.dumps(regressor)

print(type(model))
print(type(regressor))

<class 'bytes'>
<class 'sklearn.linear_model._base.LinearRegression'>


In [6]:
# Производим десериализацию
regressor_from_bytes = pickle.loads(model)
regressor_from_bytes

LinearRegression()

In [8]:
# Производим сериализацию и записываем результат в файл формата pkl
with open('pickle/myfile.pkl', 'wb') as output:
    pickle.dump(regressor, output)

In [9]:
# Производим десериализацию и извлекаем модель из файла формата pkl
with open('pickle/myfile.pkl', 'rb') as pkl_file:
    regressor_from_file = pickle.load(pkl_file)

regressor_from_file

LinearRegression()

In [19]:
# Проверяем, что все элементы массивов предсказаний совпадают между собой
print(all(regressor.predict(X) == regressor_from_bytes.predict(X)))
print(all(regressor.predict(X) == regressor_from_file.predict(X)))

True
True


In [12]:
# pickle не может сериализовать лямбда-функции
my_lambda = lambda x: x*2
with open('pickle/my_lambda.pkl', 'wb') as output:
    pickle.dump(my_lambda, output)

PicklingError: Can't pickle <function <lambda> at 0x00000250E0CE48B0>: attribute lookup <lambda> on __main__ failed

## СОХРАНЕНИЕ ПАЙПЛАЙНА

In [15]:
# Создаём пайплайн, который включает нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000250E4DF9CA0>)),
                ('Linear', LinearRegression())])

In [16]:
# Сериализуем pipeline и записываем результат в файл
with open('pickle/my_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

In [17]:
# Десериализуем pipeline из файла
with open('pickle/my_pipeline.pkl', 'rb') as pkl_file:
    loaded_pipe = pickle.load(pkl_file)

In [18]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
print(all(pipe.predict(X) == loaded_pipe.predict(X)))

True


### **Кастомные трансформеры**

наследование от двух классов: TransformerMixin и BaseEstimator

У трансформера должно быть три обязательных метода:

**__init__()** — метод, который вызывается при создании объекта данного класса. Он предназначен для инициализации исходных параметров.
Например, у трансформера для создания полиномиальных признаков PolynomialFeatures из sklearn в методе __init__() параметр degree задаёт степень полинома.

**fit()** — метод, который вызывается для «обучения» трансформера. Он должен возвращать ссылку на сам объект (self).
Например, в трансформере StandardScaler в методе fit() прописано вычисление среднего значения и стандартного отклонения в каждом столбце таблицы, переданной в качестве параметра метода fit().

**transform()** — метод, который трансформирует приходящие на вход данные. Он должен возвращать преобразованный массив данных.
Например, при вызове метода transform() у StandardScaler из sklearn внутри происходит преобразование — вычитание из каждого столбца среднего и деление результата на стандартное отклонение. Причём среднее и стандартное отклонение вычисляются заранее в методе fit().

In [ ]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''
 
    def __init__(self):
        '''
        Здесь прописывается инициализация параметров, не зависящих от данных.
        '''
        pass
 
    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''

        return self
 
    def transform(self, X):
        '''
        Здесь прописываются действия с данными.
        '''
        return X
У трансформера должно быть три 

#### **Пример трансформера**

Генерируем в данных новый признак, который является простым произведением первых трёх столбцов таблицы.

In [21]:
class MyTransformer(TransformerMixin, BaseEstimator):
    '''Шаблон кастомного трансформера'''


    def __init__(self):
        '''Здесь прописывается инициализация параметров, не зависящих от данных.'''
        pass


    def fit(self, X, y=None):
        '''
        Здесь прописывается «обучение» трансформера.
        Вычисляются необходимые для работы трансформера параметры (если они нужны).
        '''
        return self


    def transform(self, X):
        '''Здесь прописываются действия с данными.'''
        # Создаём новый столбец как произведение первых трёх
        new_column = X[:, 0] * X[:, 1] * X[:, 2]
        # Для добавления столбца в массив нужно изменить его размер на (n_rows, 1)
        new_column = new_column.reshape(X.shape[0], 1)
        # Добавляем столбец в матрицу измерений
        X = np.append(X, new_column, axis=1)
        return X

In [24]:
# Инициализируем объект класса MyTransformer (вызывается метод __init__)
custom_transformer = MyTransformer()
# Чисто формально вызываем метод fit, но у нас он ничего не делает
custom_transformer.fit(X)
# Трансформируем исходные данные (вызывается метод transform)
X_transformed = custom_transformer.transform(X)
print('Shape before transform: {}'.format(X.shape))
print('Shape after transform: {}'.format(X_transformed.shape))

Shape before transform: (442, 10)
Shape after transform: (442, 11)


In [25]:
# Создаём пайплайн, который включает Feature Engineering, нормализацию, отбор признаков и обучение модели
pipe = Pipeline([  
  ('FeatureEngineering', MyTransformer()),
  ('Scaling', MinMaxScaler()),
  ('FeatureSelection', SelectKBest(f_regression, k=5)),
  ('Linear', LinearRegression())
  ])

# Обучаем пайплайн
pipe.fit(X, y)

Pipeline(steps=[('FeatureEngineering', MyTransformer()),
                ('Scaling', MinMaxScaler()),
                ('FeatureSelection',
                 SelectKBest(k=5,
                             score_func=<function f_regression at 0x00000250E4DF9CA0>)),
                ('Linear', LinearRegression())])

In [26]:
# Сериализуем pipeline и записываем результат в файл
with open('pickle/my_new_pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

**Задание 2.5**

Десериализуйте полученный pipeline с добавленным в него кастомной трансформации из файла. Затем предскажите значение целевой переменной для наблюдения, которое описывается следующим вектором:

features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

В поле для ответа введите предсказанное значение целевой переменной, округлённое до целого числа.

In [27]:
features = np.array([[ 0.00538306, -0.04464164,  0.05954058, -0.05616605,  0.02457414, 0.05286081, -0.04340085,  0.05091436, -0.00421986, -0.03007245]])

In [33]:
with open('pickle/my_new_pipeline.pkl', 'rb') as pkl_file:
    new_pipe = pickle.load(pkl_file)

print(new_pipe.predict(features).round())

[173.]


## **БИБЛИОТЕКА JOBLIB**

In [36]:
# Производим сериализацию и сохраняем результат в файл формата .joblib
joblib.dump(regressor, 'joblib/regr.joblib')

['joblib/regr.joblib']

In [38]:
# Десериализуем модель из файла
clf_from_jobliv = joblib.load('joblib/regr.joblib') 
# Сравниваем предсказания
all(regressor.predict(X) == clf_from_jobliv.predict(X))

True